In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key
from api_keys import api_key
from api_keys import weather_api_key
#gmaps.configure(api_keys= g_key)


In [2]:
#city_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#citipy = city_url + "lompoc" + "&APPID=" + api_key
#response = requests.get(city_url)
#data = response.json()
#pprint(data)

#import openweathermapy as gmaps


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Loading in WeatherPY CSV

input_file="cities.csv"


csv=pd.read_csv(input_file)

csv.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Creating necesarry variables for Location and Humidity
locations=csv[["Lat", "Lng"]]
Humidity= csv["Humidity"]

#Define the heatmap parameters
fig=gmaps.figure()

#Create heat layer
heat_layer=gmaps.heatmap_layer(locations, weights=Humidity, dissipating= False, max_intensity=250,
                              point_radius=3)

#Add Layer
fig.add_layer(heat_layer)

#Display the map
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Personal preference
    #80> but not higher 90 
    #50< Humidity
    #Lat > 20

temp_df=csv[(csv["Max Temp"]>=80)&(csv["Max Temp"]<90)&(csv["Humidity"]<50)& (csv["Lat"]>20)]
temp_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11
15,15,albany,40,US,1558378727,45,42.65,-73.75,82.40,12.75
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88
133,133,adrar,40,DZ,1558378779,17,27.87,-0.29,89.60,28.86
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
144,144,plainview,40,US,1558378590,37,34.18,-101.71,84.99,18.34
148,148,ibra,100,OM,1558378782,41,22.69,58.55,87.18,17.40
200,200,lazaro cardenas,75,MX,1558378792,14,28.39,-105.62,88.00,21.92
227,227,tabas,100,IR,1558378798,18,33.60,56.92,89.60,9.17
341,341,raymond,40,US,1558378580,48,43.04,-71.18,84.99,12.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create df of hotel name, along with city, count, geocod(lat,long)
hotel_df=temp_df[["City","Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Names"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Names
14,constitucion,MX,23.99,-104.67,
15,albany,US,42.65,-73.75,
52,taybad,IR,34.74,60.78,
133,adrar,DZ,27.87,-0.29,
142,sarai mir,IN,26.02,82.92,
144,plainview,US,34.18,-101.71,
148,ibra,OM,22.69,58.55,
200,lazaro cardenas,MX,28.39,-105.62,
227,tabas,IR,33.60,56.92,
341,raymond,US,43.04,-71.18,


In [10]:
#Define variables
radius=8000
gtype="lodging"

#Making query calls for top Hotel

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params={"radius": radius, "types": gtype,"key": g_key}

In [12]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 14: constitucion.
Closest hotel in constitucion is 18 De Marzo.
--------------------Ending Search
Retrieving Results for Index 15: albany.
Closest hotel in albany is Hilton Albany.
--------------------Ending Search
Retrieving Results for Index 52: taybad.
Closest hotel in taybad is لی پوش jean center.
--------------------Ending Search
Retrieving Results for Index 133: adrar.
Closest hotel in adrar is s.e.t.setif entenne d'adrar.
--------------------Ending Search
Retrieving Results for Index 142: sarai mir.
Closest hotel in sarai mir is Rasoolpur Barwa.
--------------------Ending Search
Retrieving Results for Index 144: plainview.
Closest hotel in plainview is Holiday Inn Express & Suites Plainview, an IHG Hotel.
--------------------Ending Search
Retrieving Results for Index 148: ibra.
Closest hotel in ibra is مزرعة راحة النفوس المنزفة.
--------------------Ending Search
Retrieving Results for Index 200: lazaro cardenas.
Closest hotel in lazaro cardenas is Ho

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
hotel_df=hotel_df[hotel_df["Hotel Names"]!= ""]

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Names}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Plot markers on top of the heatmap
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

NameError: name 'Figure' is not defined